# ideas: would like to try to predict which trials will be successful

# step 1: define successful

# features to consider: 

- MESH terms
- country
- phase
- size
- reported_events (count)
- length
- design
- intervention category
- sponsors?
- how do they define their criteria for success?


In [16]:
import pandas as pd
import nltk

In [4]:
results = pd.read_csv('results_outcome_analysis.txt', sep='|')
results.head()

/Users/szeitlin/anaconda/envs/clinicaltrials/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RESULTS_OUTCOME_ANALYSIS_ID,OUTCOME_ID,PARAM_TYPE,DISPERSION_TYPE,CI_PERCENT,CI_LOWER_LIMIT,CI_UPPER_LIMIT,GROUPS_DESC,NON_INFERIORITY,NON_INFERIORITY_DESC,P_VALUE,P_VALUE_DESC,METHOD,METHOD_DESC,PARAM_VALUE,ESTIMATE_DESC,DISPERSION_VALUE,CI_N_SIDES,CI_UPPER_LIMIT_NA_COMMENT
0,27,20,NaN,NaN,NaN,NaN,NaN,logistic regression and mixed effects model,No,NaN,<0.05,actual calculation,"Regression, Logistic",NaN,NaN,NaN,NaN,NaN,NaN
1,28,21,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,0.18,NaN,ANOVA,NaN,NaN,NaN,NaN,NaN,NaN
2,29,26,Hazard Ratio (HR),NaN,95.00000,0.81,1.03,Adjustment performed for the following pre-spe...,No,NaN,0.12,P-value presented is not adjusted for multiple...,"Regression, Cox",NaN,0.91,NaN,NaN,2-Sided,NaN
3,30,27,Hazard Ratio (HR),NaN,95.00000,1.03,1.38,Adjustment performed for the following pre-spe...,No,NaN,0.02,P-value presented is not adjusted for multiple...,"Regression, Cox",NaN,1.19,NaN,NaN,2-Sided,NaN
4,31,89,Hazard Ratio (HR),NaN,95.00000,0.93,1.00,NaN,No,NaN,NaN,NaN,NaN,NaN,0.96,NaN,NaN,2-Sided,NaN


In [11]:
grouped = results.groupby('PARAM_TYPE').count().reset_index()
stats = grouped['PARAM_TYPE'].unique()
len(stats)

2302

In [ ]:
#aggregate these
categories = {'success rate': ['% success rate', '%success rate'],
             't-test': ['t statistic', 't value', 
                        't-value', 't-Statistic', 't-distribution', 't-stat estimated value', 't-statistic'],
             "Fisher's": ["1-sided Fisher's Exact Test", "1-sided Fisher's Exact Text"],
             }

In [45]:
#would really like to search for related terms automatically
#something like 

def apply_word_tokenize(x):
    """ Tokenize 
        :param: x (str)
        :return: token (list of str)
    >>> apply_word_tokenize('%success rate')
    ['%', 'success', 'rate']
    >>> apply_word_tokenize('t statistic')
    ['t', 'statistic']
    >>> apply_word_tokenize('t-Statistic')
    ['t', 'statistic']
    """
    words = nltk.word_tokenize(x.lower().replace('-', ' ').strip())
    return words

def find_similar(words, listofstr):
    """ find common stems in a list of strings
    :param: listofstr - raw str
    :return: stem (str)
    
    >>> find_similar(['%', 'success', 'rate'], ['%success rate', '% success rate'])
    '% success rate'
 
    >>> find_similar(['t', 'statistic'], ['t statistic', 't value', \
                        't-value', 't-Statistic', 't-distribution', 't-stat estimated value', 't-statistic'])
    't'
    """
    common = ""
    tokenized = [apply_word_tokenize(x) for x in listofstr]
    for word in words:
        if all([word in x for x in tokenized]):
            common += word + " "
    return common.strip()

In [46]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=5)

In [52]:
#Jaccard distance: 1 minus the quotient of shared N-grams and all observed N-grams
#Levenshtein distance: Minimal number of insertions, deletions and replacements needed for transforming string a into string b
nltk.__version__

'3.2.1'

In [53]:
from nltk.metrics import distance

In [57]:
from nltk.metrics.distance import jaccard_distance, edit_distance

In [56]:
jaccard_distance('%success rate', '% success rate')

AttributeError: 'str' object has no attribute 'union'

In [58]:
edit_distance('%success rate', '% success rate')

1

In [59]:
def my_jaccard_distance(label1, label2):
    """
    :param: label1, label2 (str, str)
    :return: float
    
    >>> my_jaccard_distance('%success rate', '% success rate')
    0.0
    
    """
    set1 = set([x for x in label1])
    set2 = set([x for x in label2])
    
    num_a = len(set1.union(set2))
    num_b = len(set1.intersection(set2))
    
    return (num_a - num_b)/(num_a)

In [61]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=5)